In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from tensorflow import keras
import PIL.Image as pilimg

In [3]:
mydir = "0702/"
total = mydir + "meta_total.json"
meta = pd.read_json(total)
file = np.array(meta['filename'])
emo = ["anger", "disgust", "fear", "happy", "sadness", "surprise"]

In [4]:
def reader():
    i = 0
    j = 0
    k = 550
    l = 550
    v = 50
    imlist = []
    Vimlist = []
    target = []
    Vtarget = []
    ds = 0
    while True:
        try:
            dir1 = mydir + emo[j] + "/" + file[i]
            pix = np.array(pilimg.open(dir1))
        except FileNotFoundError:
            if i - j*k >= l:
                j = j + 1
            else:
                i = i+1
                if i == 3350:
                    break
        else:
            if i - j*k < l-v:
                imlist.append(pix)
                target.append(j)
            else:
                Vimlist.append(pix)
                Vtarget.append(j)
            ds = ds +1
            i = i+1
            if i == 2250:
                i = i + l
                l = 2*l
            elif i == 3350:
                break
    X = np.array(imlist).astype('float32')
    VX = np.array(Vimlist).astype('float32')
    y = np.array(target).astype('uint8')
    Vy = np.array(Vtarget).astype('uint8')
    ts = len(X)
    vs = len(VX)
    X, y = tf.constant(X), tf.constant(y)
    VX, Vy = tf.constant(VX), tf.constant(Vy)
    train = (X, y)
    valid = (VX, Vy)
    trainset = tf.data.Dataset.from_tensor_slices(train)
    validset = tf.data.Dataset.from_tensor_slices(valid)
    return trainset, validset, ds, ts, vs

In [5]:
train_set, valid_set, dataset_size, train_size, valid_size = reader()

In [6]:
def resize(image, label):
    resized = tf.image.resize(image, [224, 224])
    return resized, label

In [7]:
def preprocess(image, label):
    resized = resize(image, label)[0]
    final = keras.applications.mobilenet_v2.preprocess_input(resized)
    return final, label

In [8]:
def mobilev2(batch_size):
    train_raw = train_set.shuffle(1000).repeat()
    train_v2 = train_raw.map(preprocess).batch(batch_size).prefetch(1)
    valid_v2 = valid_set.map(preprocess).batch(batch_size).prefetch(1)
    return train_v2, valid_v2

In [9]:
def mobilev3(batch_size):
    train_raw = train_set.shuffle(1000).repeat()
    train_v3 = train_raw.map(resize).batch(batch_size).prefetch(1)
    valid_v3 = valid_set.map(resize).batch(batch_size).prefetch(1)
    return train_v3, valid_v3

In [11]:
def compiler(model, lr=0, op="dec"):
    if op=="sgd":
        optimizer = keras.optimizers.SGD(learning_rate=lr) if lr!=0 else keras.optimizers.SGD()
    elif op=="nes":
        optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True) if lr!=0 else keras.optimizers.SGD(momentum=0.9, nesterov=True)
    elif op=="adg":
        optimizer = keras.optimizers.Adagrad(learning_rate=lr) if lr!=0 else keras.optimizers.Adagrad()
    elif op=="rms":
        optimizer = keras.optimizers.RMSprop(learning_rate=lr) if lr!=0 else keras.optimizers.RMSprop()
    elif op=="adm":
        optimizer = keras.optimizers.Adam(learning_rate=lr) if lr!=0 else keras.optimizers.Adam()
    else:
        optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [12]:
def trainer(model, batch_size, trainset, validset):
    history = model.fit(trainset, steps_per_epoch= int(train_size / batch_size), validation_data=validset, validation_steps=int(valid_size / batch_size), epochs=5)
    return model, history

In [13]:
def tflearn(V, batch_size):
    def basemodel(V):
        if V==2:
            return keras.applications.MobileNetV2(input_shape=(224,224, 3), weights="imagenet", include_top=False)
        elif V==3:
            return keras.applications.MobileNetV3Small(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
    base_model = basemodel(V)
    def tfmodel(V):
        avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
        output = keras.layers.Dense(6, activation="softmax")(avg)
        return keras.models.Model(inputs=base_model.input, outputs=output)
    model = tfmodel(V)
    def dataset(V):
        if V == 2:
            return mobilev2(batch_size)
        elif V == 3:
            return mobilev3(batch_size)
    train, valid = dataset(V)
    return base_model, model, train, valid

In [14]:
batch_size = 32
base_model, model, trainset, validset = tflearn(3, 32)

Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-07-03 12:35:19.490 tensorflow-2-6-gpu--ml-g4dn-xlarge-7eba0e151157af0bd8a25c896f03:13978 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-07-03 12:35:19.515 tensorflow-2-6-gpu--ml-g4dn-xlarge-7eba0e151157af0bd8a25c896f03:13978 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [15]:
for layer in base_model.layers:
        layer.trainable = False
model = compiler(model)
model, history = trainer(model, batch_size, trainset, validset)

Epoch 1/5
60/60 [==============================] - 7s 42ms/step - loss: 12.5144 - accuracy: 0.3615 - val_loss: 29.3217 - val_accuracy: 0.2153
Epoch 2/5
60/60 [==============================] - 1s 24ms/step - loss: 7.3257 - accuracy: 0.4417 - val_loss: 25.2399 - val_accuracy: 0.2500
Epoch 3/5
60/60 [==============================] - 1s 24ms/step - loss: 3.6632 - accuracy: 0.5396 - val_loss: 25.1866 - val_accuracy: 0.2986
Epoch 4/5
60/60 [==============================] - 1s 24ms/step - loss: 2.6126 - accuracy: 0.5536 - val_loss: 24.9734 - val_accuracy: 0.2743
Epoch 5/5
60/60 [==============================] - 1s 24ms/step - loss: 1.8330 - accuracy: 0.6062 - val_loss: 24.0566 - val_accuracy: 0.2500


In [16]:
root_logdir = os.path.join(os.curdir, "my_tb_logs")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)

In [17]:
for layer in base_model.layers:
    layer.trainable=True
optimizer = keras.optimizers.Adagrad(learning_rate=0.01, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [18]:
history = model.fit(trainset,  steps_per_epoch= int(train_size / batch_size), validation_data=validset,
                    validation_steps=int(valid_size / batch_size), epochs=100, callbacks=[tensorboard_cb])

/usr/local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/100
60/60 [==============================] - 12s 113ms/step - loss: 3.2814 - accuracy: 0.4495 - val_loss: 11.9302 - val_accuracy: 0.2569
Epoch 2/100
60/60 [==============================] - 4s 63ms/step - loss: 0.8734 - accuracy: 0.6953 - val_loss: 13.3880 - val_accuracy: 0.1944
Epoch 3/100
60/60 [==============================] - 4s 66ms/step - loss: 0.4130 - accuracy: 0.8448 - val_loss: 10.8190 - val_accuracy: 0.2535
Epoch 4/100
60/60 [==============================] - 4s 64ms/step - loss: 0.2278 - accuracy: 0.9255 - val_loss: 11.5626 - val_accuracy: 0.2326
Epoch 5/100
60/60 [==============================] - 4s 64ms/step - loss: 0.1548 - accuracy: 0.9526 - val_loss: 11.7602 - val_accuracy: 0.2431
Epoch 6/100
60/60 [==============================] - 4s 66ms/step - loss: 0.0895 - accuracy: 0.9849 - val_loss: 11.8520 - val_accuracy: 0.2188
Epoch 7/100
60/60 [==============================] - 4s 64ms/step - loss: 0.0714 - accuracy: 0.9885 - val_loss: 11.3086 - val_accuracy: 0.23

In [19]:
history = model.fit(trainset,  steps_per_epoch= int(train_size / batch_size), validation_data=validset,
                    validation_steps=int(valid_size / batch_size), epochs=100, callbacks=[tensorboard_cb])

Epoch 1/100
60/60 [==============================] - 5s 87ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 6.6381 - val_accuracy: 0.3299
Epoch 2/100
60/60 [==============================] - 4s 65ms/step - loss: 0.0051 - accuracy: 0.9995 - val_loss: 6.6268 - val_accuracy: 0.3299
Epoch 3/100
60/60 [==============================] - 4s 64ms/step - loss: 0.0056 - accuracy: 0.9995 - val_loss: 6.6313 - val_accuracy: 0.3333
Epoch 4/100
60/60 [==============================] - 4s 65ms/step - loss: 0.0047 - accuracy: 0.9995 - val_loss: 6.6466 - val_accuracy: 0.3333
Epoch 5/100
60/60 [==============================] - 4s 65ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 6.6497 - val_accuracy: 0.3299
Epoch 6/100
60/60 [==============================] - 4s 64ms/step - loss: 0.0055 - accuracy: 0.9990 - val_loss: 6.6393 - val_accuracy: 0.3333
Epoch 7/100
60/60 [==============================] - 4s 65ms/step - loss: 0.0056 - accuracy: 0.9990 - val_loss: 6.6402 - val_accuracy: 0.3333
Epoch 